# Agentic AI from Scratch — Foundations Notebook

This notebook is my sandbox to:

- Understand what an "agent" is in practice (not just theory).
- Build tiny, framework-free agents in pure Python.
- Gradually move from a simple function → to a basic agent → to agents that use tools and simple “planning”.

The emphasis here is **clarity over cleverness**. Everything should be simple enough that I can explain it back.


In [49]:
from typing import Dict, Optional, Any, List

## Simplest possible agent

In [2]:
def my_first_agent(task: str) -> dict:
    """
    A very simple 'agent'.

    Parameters
    ----------
    task : str
        A description of what you want the agent to do.
        (For now, the agent will not actually solve it, just process it.)

    Returns
    -------
    Dict[str, str]
        A dictionary with three keys:
        - 'task': the original input
        - 'reasoning': a short explanation of what the agent did internally
        - 'response': the final message from the agent
    """


    # 1. Internal "reasoning" step.
    reasoning = (
        "I received the task from the user and I am generating a very simple "
        "acknowledgement response. In later versions, this is where I will "
        "decide which tools to use or which steps to follow."
    )

    # 2. Response to return to the outside world.
    response = (
        f"I have processed your task: '{task}'. "
        "At this initial stage, I only acknowledge it and do not perform "
        "any complex actions yet."
    )

    # 3. Package everything in a structured format.
    result = {
        "task": task,
        "reasoning": reasoning,
        "response": response
    }

    return result




In [3]:
example_output = my_first_agent("Summarise the goals of this Agentic AI project.")
example_output


{'task': 'Summarise the goals of this Agentic AI project.',
 'reasoning': 'I received the task from the user and I am generating a very simple acknowledgement response. In later versions, this is where I will decide which tools to use or which steps to follow.',
 'response': "I have processed your task: 'Summarise the goals of this Agentic AI project.'. At this initial stage, I only acknowledge it and do not perform any complex actions yet."}

## Toolbox

In [11]:
# Tools

def count_words(text: str) -> int:
    """
    Simple 'tool' that count how many words are in the given texxt.
    """

    words = text.strip().split()

    return len(words)


def shout_text(text: str) -> str:
    """
    Simple 'tool' that converts the text to uppercase.
    """
    return text.upper()


Some tests

In [5]:
count_words("This is my first tiny agent")

6

In [12]:
shout_text('alamr! evacuate')

'ALAMR! EVACUATE'

## Update Agent with word count tool

In [ ]:
def word_count_agent(task: str) -> Dict[str, str]:
    """
    Upgraded agent that can now count words using the `count_words` tool.
    """
    # Behaviour

    reasoning_steps = []

    reasoning_steps.append("I received the task from the user.")

    # Search for the trigger to use the tool "count words"

    if task.lower().startswith("count words:"):
        reasoning_steps.append(
            "The task starts with 'count words:', so I will treat it as a " \
            "request to count how many words are in the provided text. " \
            "This is my cue to use the tool provided."
        )
        # Remove the 'count words:' action call from the task so that the count is only
        # on the actual text provided 
        text_to_analyse = task[len("count words:"):].strip()

        # Call the tool

        word_count = count_words(text_to_analyse)

        reasoning_steps.append(
            f"I used the 'count_words()' tool to solve the request.\n Word count: {word_count} words"
            )
        
        response = (
            f"Text analysed: '{text_to_analyse}'"
            f"Word count: {word_count}"
        )
    else:
        # Fallback behaviour
        reasoning_steps.append(
            "The task does not match the 'count words:' pattern needed to trigger the tool"
        )

        response = (
            "Please, provide a task that start with 'count words:'"
        )

    # Compile all reasoning steps
    reasoning = " ".join(reasoning_steps)

    return {
        "task": task,
        "reasoning": reasoning,
        "response": response
    }

In [10]:
# Test the updated agent

example_1 = word_count_agent("count words: this is my very first agentic tool")
example_2 = word_count_agent("summarise my project goals")

example_1, example_2


({'task': 'count words: this is my very first agentic tool',
  'reasoning': "I received the task from the user The task starts with 'count words:', so I will treat it as a request to count how many words are in the provided text. This is my cue to use the tool provided. I used the 'count_words()' tool to solve the request.\n Word count: 7 words",
  'response': "Text analysed: 'this is my very first agentic tool'Word count: 7"},
 {'task': 'summarise my project goals',
  'reasoning': "I received the task from the user The task does not match the 'count words:' pattern needed to trigger the tool",
  'response': "Please, provide a task that start with 'count words:'"})

## Command language agent

In [14]:
def simple_router_agent(task: str) -> Dict[str, str]:
    """
    An agent that can route tasks to different tools based on a simple prefix.

    Supported commands:
    - 'count words: <text>'  -> uses count_words
    - 'shout: <text>'        -> uses shout_text

    Any other task will be acknowledged but no tool will be called.
    """

    # 1. Initialise reasoning steps
    reasoning_steps = []

    reasoning_steps.append(
        "I received the task from the user."
                           )
    
    chosen_tool: Optional[str] = None
    tool_result: Optional[str] = None

    lowered_task = task.lower()

    # 2. Decide which tool (if any) to use.abs
    # If a tool is used:
    #      - extract the text
    #      - call the tool
    #      - build a response
    #    Else:
    #      - fallback response

    if lowered_task.startswith('count words:'):
        chosen_tool = 'count_words'
        reasoning_steps.append(
            "Chosen tool for the task 'count_words'."
        )
        text_to_use = task[len('count words:'):].strip()
        reasoning_steps.append(
            f"Text to analyse extracted: {text_to_use}."
        )

        word_count = count_words(text_to_use)
        tool_result = str(word_count)

        reasoning_steps.append(
            "Task processed. Counted words {word_count}"
        )
        response = (
            "Results:"
            f"Text analysed = {text_to_use}"
            f"Word count = {word_count}"
        )
    elif lowered_task.startswith('shout:'):
        chosen_tool = 'shout_text'
        reasoning_steps.append(
            "Chosen tool for the task 'shout_text'."
        )
        text_to_use = task[len('shout:'):].strip()
        reasoning_steps.append(
            f"Text to analyse extracted: {text_to_use}."
        )

        shouted = shout_text(text_to_use)
        tool_result = shouted

        reasoning_steps.append(
            "Task processed. Converted {text_to_use} to uppercase."
        )
        response = (
            "Results:"
            f"Text analysed = {text_to_use}"
            f"Uppercase = {shouted}"
        )
    else:
        reasoning_steps.append(
            "The task does not match any of my commands. Please use 'count words:' or 'shout:'"
        )
        response = (
            f"I received the task {task}, but I am unable to proccess it with my current toolset."
            "Please use 'count words:' or 'shout:'"
        )
    # 3. Join reasoning and return everything
    reasoning = ' '.join(reasoning_steps)

    return {
        "task":task,
        "chosen_tool": chosen_tool if chosen_tool is not None else "none",
        "tool_result": tool_result if tool_result is not None else "",
        "reasoning": reasoning,
        "response": response
    }

In [15]:
example_1 = simple_router_agent("count words: this is my first router agent")
example_2 = simple_router_agent("shout: this is exciting")
example_3 = simple_router_agent("please help me plan my day")

example_1, example_2, example_3


({'task': 'count words: this is my first router agent',
  'chosen_tool': 'count_words',
  'tool_result': '6',
  'reasoning': "I received the task from the user. Chosen tool for the task 'count_words'. Text to analyse extracted: this is my first router agent. Task processed. Counted words {word_count}",
  'response': 'Results:Text analysed = this is my first router agentWord count = 6'},
 {'task': 'shout: this is exciting',
  'chosen_tool': 'shout_text',
  'tool_result': 'THIS IS EXCITING',
  'reasoning': "I received the task from the user. Chosen tool for the task 'shout_text'. Text to analyse extracted: this is exciting. Task processed. Converted {text_to_use} to uppercase.",
  'response': 'Results:Text analysed = this is excitingUppercase = THIS IS EXCITING'},
 {'task': 'please help me plan my day',
  'chosen_tool': 'none',
  'tool_result': '',
  'reasoning': "I received the task from the user. The task does not match any of my commands. Please use 'count words:' or 'shout:'",
  'res

## Planner

In [33]:


Plan = Dict[str, str]

def simple_planner(task: str) -> Plan:
    """
    Planner that looks at the task and decides what action should be taken.

    It returns a 'plan' dictionary with keys:
    - 'action': one of {'count_words', 'shout', 'none'}
    - 'text': the text the chosen tool should operate on (if any)
    - 'notes': explanation of how the decision was made
    """

    lowered_task = task.lower().strip()

    # Default: do nothing special, no tool call
    action = "none"
    text = ""
    notes = "Task did not match any known pattern; no tool will be used."

    if lowered_task.startswith("count words:"):
        action = "count_words"
        # Extract text after the prefix (use original task to preserve case)
        text = task[len("count words:"):].strip()
        notes = (
            "Task starts with 'count words:', so the plan is to count how many "
            "words appear in the text after the colon."
        )

    elif lowered_task.startswith("shout:"):
        action = "shout"
        text = task[len("shout:"):].strip()
        notes = (
            "Task starts with 'shout:', so the plan is to convert the text after "
            "the colon to uppercase."
        )

    return {
        "action": action,
        "text": text,
        "notes": notes
    }


In [34]:
plan_1 = simple_planner("count words: this is a planning test")
plan_2 = simple_planner("shout: planning is fun")
plan_3 = simple_planner("just say hi")

plan_1, plan_2, plan_3


({'action': 'count_words',
  'text': 'this is a planning test',
  'notes': "Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon."},
 {'action': 'shout',
  'text': 'planning is fun',
  'notes': "Task starts with 'shout:', so the plan is to convert the text after the colon to uppercase."},
 {'action': 'none',
  'text': '',
  'notes': 'Task did not match any known pattern; no tool will be used.'})

In [ ]:

def simple_executor(plan: Plan) -> Dict[str, Any]:
    """
    Executor that takes a plan and carries it out.

    It calls the appropriate tool based on plan['action'] and returns:
    - 'action': the action it tried to perform
    - 'input_text': the text given to the tool (if any)
    - 'tool_output': the raw result from the tool (if any)
    - 'executor_notes': explanation of what was done
    """

    action = plan.get("action", "none")
    text = plan.get("text", "")
    notes = plan.get("notes", "")

    tool_output: Any = None
    executor_notes = []

    executor_notes.append(f"Received plan with action='{action}'.")
    executor_notes.append(f"Planner notes: {notes}")

    if action == "count_words":
        executor_notes.append("I will call the count_words tool.")
        tool_output = count_words(text)
        executor_notes.append(
            f"count_words returned {tool_output} for the given text."
        )

    elif action == "shout":
        executor_notes.append("I will call the shout_text tool.")
        tool_output = shout_text(text)
        executor_notes.append(
            "shout_text returned the uppercased text."
        )

    else:
        executor_notes.append(
            "Action is 'none' or unknown; I will not call any tools."
        )

    return {
        "action": action,
        "input_text": text,
        "tool_output": tool_output,
        "executor_notes": " ".join(executor_notes)
    }


In [36]:
test_plan = {
    "action": "count_words",
    "text": "this is a manual test",
    "notes": "Manual plan for testing."
}

exec_result = simple_executor(test_plan)
exec_result


{'action': 'count_words',
 'input_text': 'this is a manual test',
 'tool_output': 5,
 'executor_notes': "Received plan with action='count_words'. Planner notes: Manual plan for testing. I will call the count_words tool. count_words returned 5 for the given text."}

In [37]:
def planner_worker_agent(task: str) -> Dict[str, Any]:
    """
    Full agent that:
    1. Uses simple_planner to decide what to do.
    2. Uses simple_executor to carry out the plan.
    3. Returns a combined, structured result.
    """

    # Step 1: planning
    plan = simple_planner(task)

    # Step 2: execution
    execution_result = simple_executor(plan)

    # Step 3: Combine everything into one object
    return {
        "task": task,
        "plan": plan,
        "execution": execution_result
    }


In [38]:
res_1 = planner_worker_agent("count words: this agent is getting smarter")
res_2 = planner_worker_agent("shout: from planning to action!")
res_3 = planner_worker_agent("just chatting, no tools please")

res_1, res_2, res_3


({'task': 'count words: this agent is getting smarter',
  'plan': {'action': 'count_words',
   'text': 'this agent is getting smarter',
   'notes': "Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon."},
  'execution': {'action': 'count_words',
   'input_text': 'this agent is getting smarter',
   'tool_output': 5,
   'executor_notes': "Received plan with action='count_words'. Planner notes: Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon. I will call the count_words tool. count_words returned 5 for the given text."}},
 {'task': 'shout: from planning to action!',
  'plan': {'action': 'shout',
   'text': 'from planning to action!',
   'notes': "Task starts with 'shout:', so the plan is to convert the text after the colon to uppercase."},
  'execution': {'action': 'shout',
   'input_text': 'from planning to action!',
   'tool_output': 'FROM PLANNING TO ACTION!',
   'executor_

In [40]:
res_1["plan"], res_1["execution"]
res_2["plan"], res_2["execution"]
#res_3["plan"], res_3["execution"]

({'action': 'shout',
  'text': 'from planning to action!',
  'notes': "Task starts with 'shout:', so the plan is to convert the text after the colon to uppercase."},
 {'action': 'shout',
  'input_text': 'from planning to action!',
  'tool_output': 'FROM PLANNING TO ACTION!',
  'executor_notes': "Received plan with action='shout'. Planner notes: Task starts with 'shout:', so the plan is to convert the text after the colon to uppercase. I will call the shout_text tool. shout_text returned the uppercased text."})

## Reflection agent

In [41]:
def reflection_agent(task: str, plan: Dict[str, str], execution: Dict[str, Any]) -> Dict[str, str]:
    """
    Evaluates whether:
      - The plan appears appropriate for the task
      - The execution output makes sense
      - Any warnings should be raised

    Returns a structured evaluation with:
      - 'status': 'ok' or 'warning'
      - 'notes': explanation of the judgement
    """
    notes = []

    # Case 1: If action is "none", check if user intended something more specific.
    if plan["action"] == "none":
        notes.append(
        "Planner chose action 'none'. Checking if the task suggests a tool should have been used..."
        )
        if task.lower().startswith(("count words:", "shout:")):
            notes.append(
                "WARNING: Task looks like it matches a known pattern, "
                "but planner did not detect it. Suggest revising planner."
            )
            return {
                "status": "warning",
                "notes": " ".join(notes)
            }
        else:
            notes.append("No tool required. This seems appropriate.")
            return {
                "status": "ok",
                "notes": " ".join(notes)
            }
    # Case 2: If action is "count_words", check execution validity.
    if plan["action"] == "count_words":
        notes.append(
            "Plan requested word count; verifying execution output..."
            )
        # Execution should return an integer word count
        output = execution.get("tool_output", None)
        if isinstance(output, int) and output >= 0:
            notes.append("Word count output is valid.")
            return {
                "status": "ok",
                "notes": " ".join(notes)
            }
        else:
            notes.append("WARNING: Word count output was invalid.")
            return {
                "status": "warning",
                "notes": " ".join(notes)
            }
    # Case 3: If action is "shout", check output is uppercase.
    if plan["action"] == "shout":
        notes.append("Plan requested uppercase conversion; verifying execution output...")

        output = execution.get("tool_output", "")
        if isinstance(output, str) and output == output.upper():
            notes.append("Uppercase output is valid.")
            return {
                "status": "ok",
                "notes": " ".join(notes)
            }
        else:
            notes.append("WARNING: Uppercase conversion seems incorrect.")
            return {
                "status": "warning",
                "notes": " ".join(notes)
            }

    # Default fallback (should not happen yet)
    return {
        "status": "warning",
        "notes": "Unexpected action type. Manual inspection recommended."
    }


In [42]:
result = planner_worker_agent("count words: this reflection test is working")
result


{'task': 'count words: this reflection test is working',
 'plan': {'action': 'count_words',
  'text': 'this reflection test is working',
  'notes': "Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon."},
 'execution': {'action': 'count_words',
  'input_text': 'this reflection test is working',
  'tool_output': 5,
  'executor_notes': "Received plan with action='count_words'. Planner notes: Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon. I will call the count_words tool. count_words returned 5 for the given text."}}

In [43]:
eval_result = reflection_agent(
    task=result["task"],
    plan=result["plan"],
    execution=result["execution"]
)

eval_result

{'status': 'ok',
 'notes': 'Plan requested word count; verifying execution output... Word count output is valid.'}

In [44]:
result = planner_worker_agent("shout: hello world")
result["plan"]["action"] = "none"   # simulate planner mistake

reflection_agent(result["task"], result["plan"], result["execution"])


{'status': 'warning',
 'notes': "Planner chose action 'none'. Checking if the task suggests a tool should have been used... WARNING: Task looks like it matches a known pattern, but planner did not detect it. Suggest revising planner."}

In [45]:
bad_exec = {
    "action": "count_words",
    "input_text": "test",
    "tool_output": "not a number",
    "executor_notes": "simulated bad output"
}

reflection_agent("count words: test", {"action": "count_words"}, bad_exec)


{'status': 'warning',
 'notes': 'Plan requested word count; verifying execution output... WARNING: Word count output was invalid.'}

## Full agent pipeline

In [46]:
def super_agent(task: str) -> Dict[str, Any]:
    """
    High-level agent that:
      1. Plans what to do based on the task.
      2. Executes the plan using available tools.
      3. Evaluates the result to check if everything looks sensible.

    Returns a structured dictionary containing:
      - 'task': the original user input
      - 'plan': the plan produced by the planner
      - 'execution': the result from the executor
      - 'evaluation': the judgement from the reflection agent
    """

    # Step 1: planner + worker
    planner_worker_result = planner_worker_agent(task)

    plan = planner_worker_result["plan"]
    execution = planner_worker_result["execution"]

    # Step 2: evaluation / reflection
    evaluation = reflection_agent(
        task=task,
        plan=plan,
        execution=execution
    )

    # Step 3: combine everything
    return {
        "task": task,
        "plan": plan,
        "execution": execution,
        "evaluation": evaluation
    }

In [47]:
res_ok_count = super_agent("count words: this full pipeline seems to work well")
res_ok_shout = super_agent("shout: full pipeline online")
res_none     = super_agent("just saying hello, no special action")

res_ok_count, res_ok_shout, res_none


({'task': 'count words: this full pipeline seems to work well',
  'plan': {'action': 'count_words',
   'text': 'this full pipeline seems to work well',
   'notes': "Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon."},
  'execution': {'action': 'count_words',
   'input_text': 'this full pipeline seems to work well',
   'tool_output': 7,
   'executor_notes': "Received plan with action='count_words'. Planner notes: Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon. I will call the count_words tool. count_words returned 7 for the given text."},
  'evaluation': {'status': 'ok',
   'notes': 'Plan requested word count; verifying execution output... Word count output is valid.'}},
 {'task': 'shout: full pipeline online',
  'plan': {'action': 'shout',
   'text': 'full pipeline online',
   'notes': "Task starts with 'shout:', so the plan is to convert the text after the colon to upp

In [48]:
res_ok_count["plan"], res_ok_count["execution"], res_ok_count["evaluation"]


({'action': 'count_words',
  'text': 'this full pipeline seems to work well',
  'notes': "Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon."},
 {'action': 'count_words',
  'input_text': 'this full pipeline seems to work well',
  'tool_output': 7,
  'executor_notes': "Received plan with action='count_words'. Planner notes: Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon. I will call the count_words tool. count_words returned 7 for the given text."},
 {'status': 'ok',
  'notes': 'Plan requested word count; verifying execution output... Word count output is valid.'})

## Iterative / looping agent

In [50]:
def iterative_super_agent(task: str, max_attempts: int = 3) -> Dict[str, Any]:
    """
    Iterative version of the full agent pipeline.

    It:
      1. Repeats the (planner -> worker -> evaluator) cycle up to `max_attempts` times.
      2. Stops early if the evaluation status is 'ok'.
      3. Stores a history of all attempts.

    Returns:
      {
        'task': ...,
        'attempts': [
            {
                'attempt': int,
                'plan': {...},
                'execution': {...},
                'evaluation': {...}
            },
            ...
        ],
        'final_status': 'ok' or 'warning'
      }
    """

    attempts: List[Dict[str, Any]] = []

    for attempt_idx in range(1, max_attempts + 1):
        # Run the standard pipeline once
        planner_worker_result = planner_worker_agent(task)
        plan = planner_worker_result["plan"]
        execution = planner_worker_result["execution"]

        evaluation = reflection_agent(
            task=task,
            plan=plan,
            execution=execution
        )

        # Store this attempt
        attempts.append({
            "attempt": attempt_idx,
            "plan": plan,
            "execution": execution,
            "evaluation": evaluation
        })

        # Check if evaluation is satisfied
        if evaluation.get("status") == "ok":
            # Stop early if everything looks good
            break

        # If it's a warning, in a real system we might:
        # - adjust the plan
        # - modify the task
        # - ask the user for clarification
        # For now, we simply try again with the same task until max_attempts is reached.

    final_status = attempts[-1]["evaluation"]["status"]

    return {
        "task": task,
        "attempts": attempts,
        "final_status": final_status
    }


In [51]:
iter_res_ok = iterative_super_agent("count words: this iterative agent seems fine")
iter_res_ok


{'task': 'count words: this iterative agent seems fine',
 'attempts': [{'attempt': 1,
   'plan': {'action': 'count_words',
    'text': 'this iterative agent seems fine',
    'notes': "Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon."},
   'execution': {'action': 'count_words',
    'input_text': 'this iterative agent seems fine',
    'tool_output': 5,
    'executor_notes': "Received plan with action='count_words'. Planner notes: Task starts with 'count words:', so the plan is to count how many words appear in the text after the colon. I will call the count_words tool. count_words returned 5 for the given text."},
   'evaluation': {'status': 'ok',
    'notes': 'Plan requested word count; verifying execution output... Word count output is valid.'}}],
 'final_status': 'ok'}

In [52]:
# Quick hack to see multiple attempts:
# Call the iterative agent with a 'normal' task,
# but artificially force the evaluation to warning by editing after the fact.

iter_res = iterative_super_agent("just saying hi, no tools")
iter_res


{'task': 'just saying hi, no tools',
 'attempts': [{'attempt': 1,
   'plan': {'action': 'none',
    'text': '',
    'notes': 'Task did not match any known pattern; no tool will be used.'},
   'execution': {'action': 'none',
    'input_text': '',
    'tool_output': None,
    'executor_notes': "Received plan with action='none'. Planner notes: Task did not match any known pattern; no tool will be used. Action is 'none' or unknown; I will not call any tools."},
   'evaluation': {'status': 'ok',
    'notes': "Planner chose action 'none'. Checking if the task suggests a tool should have been used... No tool required. This seems appropriate."}}],
 'final_status': 'ok'}

# === End of Stage 1 ===